* Verified on VGG clusters.
* Used Python 2.7 installation at `/users/ruthfong/anaconda2/`
* Using CPU Caffe installation at `export PYTHONPATH=$PYTHONPATH:/users/ruthfong/caffe-1.0/python`

# Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from skimage.transform import resize


import caffe
import torch

from utils import get_finetune_model
from caffe_transforms import get_caffe_transform

# Definitions

In [2]:
# Set paths.

caffe_models_dir = "/scratch/shared/slow/ruthfong/pointing_game/Jiaming_All_Caffe_Models"
pytorch_models_dir = "/scratch/shared/slow/ruthfong/pointing_game"
image_dir = "/users/ruthfong/feature_attribution/images"

img_scale = 224

image_path = os.path.join(image_dir, "doberman.png")

# Helper Functions

In [3]:
def get_caffe_model_paths(arch, dataset):
    if arch == 'vgg16':
        pass
    elif arch == 'resnet50':
        pass
    else:
        assert False

    if 'voc' in dataset:
        if arch == 'vgg16':
            model_dir = os.path.join(caffe_models_dir, 'voc', 'VGG16')
            proto_name = 'deploy_fcn.prototxt'
            model_name = 'VGG16VOC07.caffemodel'
        elif arch == 'resnet50':
            model_dir = os.path.join(caffe_models_dir, 'voc', 'ResNet50')
            proto_name = 'deploy.prototxt'
            model_name = '_iter_230000.caffemodel'
        else:
            assert False
        last_layer = 'fc8-20-conv'
    elif 'coco' in dataset:
        if arch == 'vgg16':
            model_dir = os.path.join(caffe_models_dir, 'coco', 'VGG16')
            proto_name = 'deploy_fcn.prototxt'
            model_name = 'VGG16COCO.caffemodel'
        elif arch == 'resnet50':
            model_dir = os.path.join(caffe_models_dir, 'coco', 'ResNet50')
            proto_name = 'resnet50-deploy_Jianming.prototxt'
            model_name = 'resnet50-coco.caffemodel'
        else:
            assert False
        last_layer = 'fc8-80-conv'
    else:
        assert False

    return os.path.join(model_dir, proto_name), os.path.join(model_dir, model_name), last_layer


def get_ILSVRC_net_transformer(net):
    transformer = caffe.io.Transformer({'data':net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.array([103.939, 116.779, 123.68]))
    transformer.set_raw_scale('data', 255)
    transformer.set_channel_swap('data', (2,1,0))
    return transformer


def get_ILSVRC_net_transformer_with_shape(shape):
    transformer = caffe.io.Transformer({'data':shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.array([103.939, 116.779, 123.68]))
    transformer.set_raw_scale('data', 255)
    transformer.set_channel_swap('data', (2,1,0))
    return transformer

In [4]:
def get_pytorch_model_path(arch, dataset):
    if 'voc' in dataset:
        if arch == 'vgg16':
            return os.path.join(pytorch_models_dir, 'vgg16_pascal07_Jianming.pth.tar')
        elif arch == 'resnet50':
            return os.path.join(pytorch_models_dir, 'resnet50_pascal07_Jianming_pytorch.pth.tar')
        else:
            assert False
    elif 'coco' in dataset:
        if arch == 'vgg16':
            return os.path.join(pytorch_models_dir, 'vgg16_mscoco_Jianming.pth.tar')
        elif arch == 'resnet50':
            return os.path.join(pytorch_models_dir, 'resnet50_mscoco_Jianming_pytorch.pth.tar')
        else:
            assert False
    else:
        assert False

In [8]:
def get_caffe_output(arch, dataset):
    # Initialize caffe model.
    proto_path, model_path, last_layer = get_caffe_model_paths(arch=arch, dataset=dataset)
    caffe_model = caffe.Net(proto_path, model_path, caffe.TEST)

    # Load image.
    orig_img = caffe.io.load_image(image_path)
    # print(orig_img.shape)

    # Resize image.
    min_dim = min(orig_img.shape[:2])
    new_size = (int(orig_img.shape[0]*img_scale/float(min_dim)), int(orig_img.shape[1]*img_scale/float(min_dim)))
    img = resize(orig_img, new_size)
    # print(img.shape)

    # Change input shape.
    caffe_model.blobs['data'].reshape(1, 3, img.shape[0], img.shape[1])

    # Get caffe transformer.
    transformer = get_ILSVRC_net_transformer_with_shape((1, 3, img.shape[0], img.shape[1]))

    # Set input as preprocessed image.
    transformed_img = transformer.preprocess('data', img)
    caffe_model.blobs['data'].data[...] = transformed_img

    # Get output.
    out = caffe_model.forward(end=last_layer)
    return transformed_img, out


def get_pytorch_output(arch, dataset):
    # Get PyTorch model.
    checkpoint_path = get_pytorch_model_path(arch=arch, dataset=dataset)
    pytorch_model = get_finetune_model(arch=arch,
                                       dataset=dataset,
                                       converted_caffe=True,
                                       checkpoint_path=checkpoint_path,
                                       convert_to_fully_convolutional=True,
                                       final_gap_layer=True)

    # Get Caffe transformation.
    transform = get_caffe_transform(img_scale)

    # Transform image.
    orig_img_torch = Image.open(image_path).convert('RGB')
    img_torch = transform(orig_img_torch)

    # Run network.
    out_pytorch = pytorch_model(img_torch.unsqueeze(0))

    return img_torch, out_pytorch

# Verify the equivalency of Caffe and PyTorch models.

In [11]:
arches = ['vgg16', 'resnet50']
datasets = ['voc_2007', 'coco_2014']

for arch in arches:
    for dataset in datasets:
        print('Checking %s %s...' % (arch, dataset))
        caffe_img, caffe_out = get_caffe_output(arch, dataset)
        pytorch_img, pytorch_out = get_pytorch_output(arch, dataset)

        # Check that pre-processed image is the same.
        assert np.sum(caffe_img - pytorch_img.numpy()) < 1e-5

        # Check that output is nearly the same (looks like small floating point differences).
        assert np.mean(caffe_out['fc8'].squeeze() - pytorch_out.data.numpy().squeeze()) < 1e-5
        print('Correct!\n')

Checking vgg16 voc_2007...
Correct!

Checking vgg16 coco_2014...
Correct!

Checking resnet50 voc_2007...
Correct!

Checking resnet50 coco_2014...
Correct!

